In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
    
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

D:\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['expect', 'from', 'subject', 're', 'use', 'iss', 'issue', 'bug', 'error', 'test', 'remove', 'event', 'result', 'enable', 'support'])

# NLTK Stop words - Noun
from nltk.corpus import stopwords
stop_words_noun = stopwords.words('english')
stop_words_noun.extend(['iss', 'value', 'case', 'from', 'subject', 're', 'show', 'crash', 'issue', 'error', 'support', 'bug', 'area','default'])

# NLTK Stop words - Verb
from nltk.corpus import stopwords
stop_words_verb = stopwords.words('english')
stop_words_verb.extend(['from', 'subject', 're', 'edu', 'use','iss','issue', 'iss-metro', 'ldp', 'iss_customiz'])

In [4]:
# Import Dataset
df = pd.read_csv('CSR_title_text.csv', error_bad_lines=False);
df.head()

,CSR_no,SR_title
0,143979,When will earfcns greater than 65535 be suppor...
1,144595,SR periodicity
2,144653,LTE Stack Issue with CLI
3,146023,L2 crash (Segmentation fault) when originating...
4,146100,L2 crash after pumping UL data from a UE.


In [5]:
# Convert to list
data = df.SR_title.values.tolist()
df.dtypes

CSR_no       int64
SR_title    object
dtype: object

In [6]:
# Remove special characters
data = [re.sub(r'[@#\$%&\*\(\)\<\>\?\'\";:\]\[-]', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

pprint(data[:1])

['When will earfcns greater than 65535 be supported ']


In [7]:
#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['when', 'will', 'earfcns', 'greater', 'than', 'be', 'supported']]


In [8]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def remove_stopwords_noun(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_noun] for doc in texts]

def remove_stopwords_verb(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_verb] for doc in texts]

In [9]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
len(data_words)
len(data_words)
print(data_words_nostops[13:14])

data_words_nostops_noun = remove_stopwords_noun(data_words)
len(data_words)
len(data_words_nostops_noun)
print(data_words_nostops[13:14])

data_words_nostops_verb = remove_stopwords_verb(data_words)
len(data_words)
len(data_words_nostops_verb)
print(data_words_nostops[1:2])

[['rohc', 'working', 'crash', 'happened', 'rohc']]
[['rohc', 'working', 'crash', 'happened', 'rohc']]
[['sr', 'periodicity']]


In [11]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_noun = lemmatization(data_words_nostops_noun, allowed_postags=['NOUN'])
data_lemmatized_noun = [x for x in data_lemmatized_noun if x != []]

data_lemmatized_verb = lemmatization(data_words_nostops_verb, allowed_postags=['VERB'])
data_lemmatized_verb = [x for x in data_lemmatized_verb if x != []]

In [12]:
# Remove Stop Words from lemmatized words
data_lemmatized = remove_stopwords(data_lemmatized)
len(data_words)
len(data_lemmatized)
print(data_lemmatized[13:14])

data_lemmatized_noun = remove_stopwords_noun(data_lemmatized_noun)
len(data_words)
len(data_lemmatized_noun)
print(data_lemmatized_noun[13:14])

data_lemmatized_verb = remove_stopwords_verb(data_lemmatized_verb)
len(data_words)
len(data_lemmatized_verb)
print(data_lemmatized_verb[13:14])

[['rohc', 'work', 'crash', 'happen', 'rohc']]
[['build', 'package']]
[['feature', 'require']]


In [13]:
#Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

id2word_noun = corpora.Dictionary(data_lemmatized_noun)

id2word_verb = corpora.Dictionary(data_lemmatized_verb)

In [14]:
# Create Corpus
texts = data_lemmatized

texts_noun = data_lemmatized_noun

texts_verb = data_lemmatized_verb

In [15]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

corpus_noun = [id2word_noun.doc2bow(text) for text in texts_noun]

corpus_verb = [id2word_verb.doc2bow(text) for text in texts_verb]

In [16]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[13:14]]

[[(id2word[id], freq) for id, freq in cp] for cp in corpus_noun[13:14]]

[[(id2word[id], freq) for id, freq in cp] for cp in corpus_verb[13:14]]


[[('enodeb', 1), ('exchange', 1)]]

In [17]:
#Building the Topic Model
# Build LDA model for whole text
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
#Building the Topic Model
# Build LDA model for whole noun
lda_model_noun = gensim.models.ldamodel.LdaModel(corpus=corpus_noun,
                                           id2word=id2word_noun,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
#Building the Topic Model
# Build LDA model for whole verb
lda_model_verb = gensim.models.ldamodel.LdaModel(corpus=corpus_verb,
                                           id2word=id2word_verb,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)


In [31]:
#View the topics in LDA model
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_words=30))
doc_lda = lda_model[corpus]

[(0,
  '0.101*"eoam" + 0.069*"dev" + 0.038*"remote" + 0.038*"behavior" + '
  '0.029*"symbolperiod" + 0.017*"work" + 0.014*"node" + 0.013*"tunnel" + '
  '0.013*"bfd" + 0.012*"mac" + 0.011*"reload" + 0.011*"go" + 0.010*"update" + '
  '0.010*"path" + 0.009*"shut" + 0.009*"fault" + 0.008*"type" + '
  '0.008*"failure" + 0.008*"primary" + 0.008*"switch" + 0.008*"segmentation" + '
  '0.007*"remain" + 0.007*"end" + 0.007*"ring" + 0.007*"code" + '
  '0.006*"present" + 0.006*"even" + 0.006*"user" + 0.006*"protocol" + '
  '0.006*"status"'),
 (1,
  '0.073*"redmine" + 0.046*"route" + 0.032*"ospf" + 0.027*"interface" + '
  '0.023*"ospfv" + 0.022*"configure" + 0.020*"tlvs" + 0.019*"ipv" + '
  '0.018*"isis" + 0.016*"configuration" + 0.016*"fail" + 0.015*"fsu" + '
  '0.013*"router" + 0.013*"entry" + 0.012*"vpn" + 0.012*"crash" + '
  '0.011*"default" + 0.010*"ldp" + 0.010*"mpls" + 0.009*"area" + 0.008*"set" + '
  '0.008*"traffic" + 0.008*"static" + 0.008*"learn" + 0.007*"peer" + '
  '0.007*"new" + 0.007

In [30]:
#View the topics in LDA noun model 
# Print the Keyword in the 10 topics
pprint(lda_model_noun.print_topics(num_words=30))
doc_lda_noun = lda_model_noun[corpus_noun]

[(0,
  '0.057*"interface" + 0.055*"ospf" + 0.048*"command" + 0.041*"ip" + '
  '0.038*"drop" + 0.026*"router" + 0.019*"packet" + 0.018*"blocker" + '
  '0.016*"time" + 0.015*"traffic" + 0.014*"question" + 0.013*"output" + '
  '0.013*"peer" + 0.011*"tlv" + 0.011*"module" + 0.011*"shutdown" + '
  '0.010*"igs" + 0.010*"information" + 0.009*"link" + 0.009*"lsa" + '
  '0.009*"user" + 0.009*"fault" + 0.009*"ig" + 0.009*"behavior" + '
  '0.009*"config" + 0.008*"segmentation" + 0.008*"ecmp" + 0.008*"sync" + '
  '0.007*"code" + 0.007*"memory"'),
 (1,
  '0.119*"redmine" + 0.077*"route" + 0.052*"port" + 0.030*"vlan" + '
  '0.029*"isis" + 0.024*"session" + 0.023*"entry" + 0.022*"ipv" + 0.021*"fsu" '
  '+ 0.020*"vpn" + 0.019*"address" + 0.018*"scenario" + 0.015*"ospfv" + '
  '0.014*"igmp" + 0.013*"table" + 0.013*"qos" + 0.012*"number" + 0.011*"acl" + '
  '0.011*"svt" + 0.010*"mrouter" + 0.010*"csr" + 0.010*"mac" + 0.009*"request" '
  '+ 0.008*"portchannel" + 0.008*"routemap" + 0.007*"bgp" + 0.006*"co

In [29]:
#View the topics in LDA verb model 
# Print the Keyword in the 10 topics
pprint(lda_model_verb.print_topics(num_words=30))
doc_lda_verb = lda_model_verb[corpus_verb]

[(0,
  '0.121*"fail" + 0.103*"send" + 0.081*"display" + 0.051*"go" + 0.038*"instal" '
  '+ 0.034*"remain" + 0.033*"create" + 0.032*"would" + 0.026*"give" + '
  '0.015*"log" + 0.015*"stp" + 0.014*"snmpv" + 0.014*"follow" + 0.011*"stop" + '
  '0.010*"rtm" + 0.009*"return" + 0.008*"schedule" + 0.008*"recover" + '
  '0.007*"leave" + 0.007*"help" + 0.007*"call" + 0.006*"seem" + '
  '0.006*"include" + 0.006*"max" + 0.005*"contain" + 0.005*"present" + '
  '0.005*"link" + 0.005*"respond" + 0.005*"flush" + 0.005*"flap"'),
 (1,
  '0.089*"enable" + 0.085*"change" + 0.057*"work" + 0.048*"observe" + '
  '0.032*"reload" + 0.025*"receive" + 0.024*"miss" + 0.022*"rout" + '
  '0.021*"occur" + 0.021*"perform" + 0.020*"relate" + 0.019*"forward" + '
  '0.017*"take" + 0.017*"rip" + 0.016*"disable" + 0.016*"regard" + 0.013*"ig" '
  '+ 0.013*"transmit" + 0.012*"elect" + 0.012*"establish" + 0.012*"expire" + '
  '0.011*"stay" + 0.011*"happen" + 0.009*"accept" + 0.009*"sync" + '
  '0.008*"stick" + 0.008*"join" 

In [23]:
#Compute Model Perplexity and Coherence Score
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Perplexity
print('\nPerplexity: ', lda_model_verb.log_perplexity(corpus_verb))  # a measure of how good the model is. lower the better.

# Compute Perplexity
print('\nPerplexity: ', lda_model_noun.log_perplexity(corpus_noun))  # a measure of how good the model is. lower the better. 


Perplexity:  -7.706076333241437

Perplexity:  -6.354237855077625

Perplexity:  -7.3751259710863595


In [24]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda_noun = CoherenceModel(model=lda_model_noun, texts=data_lemmatized_noun, dictionary=id2word_noun, coherence='c_v')
coherence_lda_noun = coherence_model_lda_noun.get_coherence()
print('\nCoherence Score noun: ', coherence_lda_noun)

coherence_model_lda_verb = CoherenceModel(model=lda_model_verb, texts=data_lemmatized_verb, dictionary=id2word_verb, coherence='c_v')
coherence_lda_verb = coherence_model_lda_verb.get_coherence()
print('\nCoherence Score verb: ', coherence_lda_verb)


Coherence Score:  0.3876856847583213

Coherence Score noun:  0.46586660259100254

Coherence Score verb:  0.6207764547543433


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

D:\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.349064  0.028150       1        1  33.383743
3     -0.149508  0.296049       2        1  25.157654
2     -0.118823 -0.216909       3        1  21.577021
0     -0.080733 -0.107290       4        1  19.881577, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
4380  Default   946.000000           eoam   946.000000  30.0000  30.0000
1345  Default  1145.000000        redmine  1145.000000  29.0000  29.0000
4318  Default   643.000000            dev   643.000000  28.0000  28.0000
318   Default   598.000000       loopback   598.000000  27.0000  27.0000
503   Default   730.000000          route   730.000000  26.0000  26.0000
78    Default   455.000000          state   455.000000  25.0000  25.0000
58    Default   447.000000           show   447.000000  24.0000  24.0000
237   Default   390.000000        command   390.000000  23.0000  23.0000
198   Default   425.000000           port   425.000000  22.0000  22.0000
729   Default   359.000000         remote   359.000000  21.0000  21.0000
664   Default   352.000000       behavior   352.000000  20.0000  20.0000
271   Default   509.000000           ospf   509.000000  19.0000  19.0000
57    Default   347.000000             ip   347.000000  18.0000  18.0000
194   Default   431.000000      interface   431.000000  17.0000  17.0000
427   Default   312.000000           drop   312.000000  16.0000  16.0000
1510  Default   367.000000          ospfv   367.000000  15.0000  15.0000
345   Default   278.000000           vlan   278.000000  14.0000  14.0000
253   Default   347.000000      configure   347.000000  13.0000  13.0000
30    Default   267.000000         change   267.000000  12.0000  12.0000
4573  Default   305.000000   symbolperiod   305.000000  11.0000  11.0000
415   Default   304.000000            ipv   304.000000  10.0000  10.0000
136   Default   208.000000           send   208.000000   9.0000   9.0000
339   Default   287.000000           isis   287.000000   8.0000   8.0000
166   Default   197.000000        session   197.000000   7.0000   7.0000
4342  Default   323.000000           tlvs   323.000000   6.0000   6.0000
31    Default   257.000000  configuration   257.000000   5.0000   5.0000
327   Default   181.000000        display   181.000000   4.0000   4.0000
84    Default   255.000000           fail   255.000000   3.0000   3.0000
27    Default   161.000000           work   161.000000   2.0000   2.0000
227   Default   176.000000            get   176.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
729    Topic4   358.325745         remote   359.066864   1.6133  -3.2621
664    Topic4   351.267426       behavior   352.007965   1.6133  -3.2820
4318   Topic4   642.383545            dev   643.965637   1.6129  -2.6783
27     Topic4   160.382538           work   161.039902   1.6113  -4.0660
93     Topic4   132.075134           node   132.723785   1.6105  -4.2602
209    Topic4   124.830208         tunnel   125.469368   1.6103  -4.3166
499    Topic4   124.811134            bfd   125.466537   1.6101  -4.3167
385    Topic4   110.891045            mac   111.575157   1.6092  -4.4350
350    Topic4   105.691154         reload   106.350754   1.6092  -4.4830
193    Topic4   104.221359             go   104.896126   1.6089  -4.4970
217    Topic4    96.760796           path    97.403557   1.6088  -4.5713
145    Topic4    97.396469         update    98.068047   1.6085  -4.5647
298    Topic4    87.566978           shut    88.220741   1.6079  -4.6711
10     Topic4    87.029236          fault    87.699783   1.6077  -4.6773
114    Topic4    77.717461        failure    78.376389   1.6069  -4.7904
338    Topic4    76.835922         switch    77.495049   1.6068  -4.8019
405    Topic4    79.303886           type    79.995461  

In [26]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_noun = pyLDAvis.gensim.prepare(lda_model_noun, corpus_noun, id2word_noun)
vis_noun

D:\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.335826  0.083949       1        1  28.841730
0      0.196114  0.269925       2        1  28.611609
2      0.070570 -0.193407       3        1  24.516253
3      0.069142 -0.160467       4        1  18.030409, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
852   Default  991.000000        redmine  991.000000  30.0000  30.0000
331   Default  643.000000          route  643.000000  29.0000  29.0000
136   Default  469.000000      interface  469.000000  28.0000  28.0000
179   Default  452.000000           ospf  452.000000  27.0000  27.0000
138   Default  433.000000           port  433.000000  26.0000  26.0000
161   Default  397.000000        command  397.000000  25.0000  25.0000
21    Default  309.000000  configuration  309.000000  24.0000  24.0000
40    Default  340.000000             ip  340.000000  23.0000  23.0000
53    Default  302.000000          state  302.000000  22.0000  22.0000
308   Default  318.000000           drop  318.000000  21.0000  21.0000
231   Default  251.000000           vlan  251.000000  20.0000  20.0000
226   Default  243.000000           isis  243.000000  19.0000  19.0000
801   Default  166.000000           card  166.000000  18.0000  18.0000
797   Default  164.000000     regression  164.000000  17.0000  17.0000
240   Default  215.000000         router  215.000000  16.0000  16.0000
118   Default  197.000000        session  197.000000  15.0000  15.0000
233   Default  193.000000          entry  193.000000  14.0000  14.0000
142   Default  169.000000           mpls  169.000000  13.0000  13.0000
2936  Default  177.000000            dev  177.000000  12.0000  12.0000
37    Default  161.000000           test  161.000000  11.0000  11.0000
281   Default  183.000000            ipv  183.000000  10.0000  10.0000
41    Default  130.000000          query  130.000000   9.0000   9.0000
746   Default  178.000000            fsu  178.000000   8.0000   8.0000
144   Default  165.000000            vpn  165.000000   7.0000   7.0000
147   Default  140.000000         tunnel  140.000000   6.0000   6.0000
125   Default  158.000000         packet  158.000000   5.0000   5.0000
63    Default  154.000000        address  154.000000   4.0000   4.0000
202   Default  134.000000           mode  134.000000   3.0000   3.0000
811   Default  152.000000        blocker  152.000000   2.0000   2.0000
513   Default  108.000000        version  108.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2      Topic4  107.555023            cli  108.224495   1.7069  -3.8783
513    Topic4  107.953972        version  108.660347   1.7066  -3.8746
689    Topic4   96.246231        feature   96.912514   1.7062  -3.9894
502    Topic4   93.391731           host   94.117714   1.7054  -4.0195
321    Topic4   89.007629          event   89.733986   1.7050  -4.0676
79     Topic4   78.569725        failure   79.218262   1.7049  -4.1923
273    Topic4   78.173790           type   78.834457   1.7047  -4.1974
47     Topic4   77.795860         system   78.471283   1.7045  -4.2022
249    Topic4   73.249832       priority   73.928452   1.7039  -4.2624
76     Topic4   68.814072        message   69.453522   1.7039  -4.3249
988    Topic4   71.261612            erp   71.983421   1.7030  -4.2899
405    Topic4   66.770767            mib   67.449532   1.7030  -4.3550
310    Topic4   63.543571         switch   64.195717   1.7029  -4.4046
549    Topic4   66.995201       interval   67.689911   1.7028  -4.3517
407    Topic4   54.663216          timer   55.302200   1.7015  -4.5551
302    Topic4   51.299686            vrf   51.972729   1.7001  -4.6186
366    Topic4   55.426781        console   56.156834   1.7000  -4.5412
28     Topic4   49.305485        network   49.956276   1.7000  -4.6583
598    Topic

In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_verb = pyLDAvis.gensim.prepare(lda_model_verb, corpus_verb, id2word_verb)
vis_verb

D:\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.339105  0.004150       1        1  34.058609
1     -0.117760  0.283947       2        1  25.171539
2     -0.108953 -0.101556       3        1  20.396271
0     -0.112392 -0.186541       4        1  20.373579, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
20    Default  247.000000        fail  247.000000  30.0000  30.0000
94    Default  210.000000        send  210.000000  29.0000  29.0000
54    Default  308.000000   configure  308.000000  28.0000  28.0000
7     Default  224.000000      enable  224.000000  27.0000  27.0000
71    Default  214.000000      change  214.000000  26.0000  26.0000
70    Default  165.000000     display  165.000000  25.0000  25.0000
44    Default  245.000000        show  245.000000  24.0000  24.0000
66    Default  155.000000      remove  155.000000  23.0000  23.0000
353   Default  205.000000       snoop  205.000000  22.0000  22.0000
45    Default  203.000000         get  203.000000  21.0000  21.0000
364   Default  133.000000      repeat  133.000000  20.0000  20.0000
6     Default  142.000000        work  142.000000  19.0000  19.0000
74    Default  122.000000     observe  122.000000  18.0000  18.0000
39    Default  104.000000          go  104.000000  17.0000  17.0000
32    Default   94.000000        come   94.000000  16.0000  16.0000
56    Default  137.000000     execute  137.000000  15.0000  15.0000
88    Default  126.000000       learn  126.000000  14.0000  14.0000
247   Default   78.000000      instal   78.000000  13.0000  13.0000
83    Default  110.000000         set  110.000000  12.0000  12.0000
30    Default   73.000000      update   73.000000  11.0000  11.0000
79    Default   81.000000      reload   81.000000  10.0000  10.0000
52    Default   69.000000      remain   69.000000   9.0000   9.0000
27    Default   67.000000      create   67.000000   8.0000   8.0000
84    Default   65.000000       would   65.000000   7.0000   7.0000
131   Default   64.000000         see   64.000000   6.0000   6.0000
21    Default   62.000000     receive   62.000000   5.0000   5.0000
26    Default   53.000000       reset   53.000000   4.0000   4.0000
116   Default   53.000000        give   53.000000   3.0000   3.0000
38    Default   52.000000        need   52.000000   2.0000   2.0000
65    Default   80.000000        shut   80.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
94     Topic4  209.607605        send  210.287003   1.5877  -2.2728
70     Topic4  165.152893     display  165.841370   1.5868  -2.5111
39     Topic4  103.718712          go  104.406067   1.5843  -2.9763
247    Topic4   77.772881      instal   78.489120   1.5818  -3.2642
27     Topic4   66.837532      create   67.499596   1.5811  -3.4157
84     Topic4   64.738205       would   65.392242   1.5809  -3.4477
52     Topic4   68.473885      remain   69.169006   1.5808  -3.3916
116    Topic4   52.635235        give   53.311214   1.5782  -3.6546
223    Topic4   30.000540         log   30.685944   1.5683  -4.2168
390    Topic4   29.662085         stp   30.354729   1.5678  -4.2281
210    Topic4   29.154203      follow   29.848150   1.5674  -4.2454
24     Topic4   23.002338        stop   23.669821   1.5623  -4.4824
381    Topic4   20.368473         rtm   21.135471   1.5540  -4.6040
265    Topic4   17.520008      return   18.224184   1.5515  -4.7547
894    Topic4   29.320644       snmpv   30.629206   1.5473  -4.2397
259    Topic4   13.375455        call   14.021694   1.5437  -5.0246
123    Topic4   13.513565        help   14.172295   1.5433  -5.0143
255    Topic4   15.585559     recover   16.356819   1.5426  -4.8717
286    Topic4   13.663494       leave   14.341748   1.5425  -5.0033
451    Topic4   12.089462     include   12.768143   1.5363  -5.1257
51     Topic4   12.575818    

In [32]:
lda_model_noun.print_topics(num_words=30)

[(0,
  '0.057*"interface" + 0.055*"ospf" + 0.048*"command" + 0.041*"ip" + 0.038*"drop" + 0.026*"router" + 0.019*"packet" + 0.018*"blocker" + 0.016*"time" + 0.015*"traffic" + 0.014*"question" + 0.013*"output" + 0.013*"peer" + 0.011*"tlv" + 0.011*"module" + 0.011*"shutdown" + 0.010*"igs" + 0.010*"information" + 0.009*"link" + 0.009*"lsa" + 0.009*"user" + 0.009*"fault" + 0.009*"ig" + 0.009*"behavior" + 0.009*"config" + 0.008*"segmentation" + 0.008*"ecmp" + 0.008*"sync" + 0.007*"code" + 0.007*"memory"'),
 (1,
  '0.119*"redmine" + 0.077*"route" + 0.052*"port" + 0.030*"vlan" + 0.029*"isis" + 0.024*"session" + 0.023*"entry" + 0.022*"ipv" + 0.021*"fsu" + 0.020*"vpn" + 0.019*"address" + 0.018*"scenario" + 0.015*"ospfv" + 0.014*"igmp" + 0.013*"table" + 0.013*"qos" + 0.012*"number" + 0.011*"acl" + 0.011*"svt" + 0.010*"mrouter" + 0.010*"csr" + 0.010*"mac" + 0.009*"request" + 0.008*"portchannel" + 0.008*"routemap" + 0.007*"bgp" + 0.006*"configure" + 0.006*"redistribute" + 0.006*"deadlock" + 0.006*"

In [33]:
X = lda_model_noun.print_topics(num_words=30)

In [34]:
X

[(0,
  '0.057*"interface" + 0.055*"ospf" + 0.048*"command" + 0.041*"ip" + 0.038*"drop" + 0.026*"router" + 0.019*"packet" + 0.018*"blocker" + 0.016*"time" + 0.015*"traffic" + 0.014*"question" + 0.013*"output" + 0.013*"peer" + 0.011*"tlv" + 0.011*"module" + 0.011*"shutdown" + 0.010*"igs" + 0.010*"information" + 0.009*"link" + 0.009*"lsa" + 0.009*"user" + 0.009*"fault" + 0.009*"ig" + 0.009*"behavior" + 0.009*"config" + 0.008*"segmentation" + 0.008*"ecmp" + 0.008*"sync" + 0.007*"code" + 0.007*"memory"'),
 (1,
  '0.119*"redmine" + 0.077*"route" + 0.052*"port" + 0.030*"vlan" + 0.029*"isis" + 0.024*"session" + 0.023*"entry" + 0.022*"ipv" + 0.021*"fsu" + 0.020*"vpn" + 0.019*"address" + 0.018*"scenario" + 0.015*"ospfv" + 0.014*"igmp" + 0.013*"table" + 0.013*"qos" + 0.012*"number" + 0.011*"acl" + 0.011*"svt" + 0.010*"mrouter" + 0.010*"csr" + 0.010*"mac" + 0.009*"request" + 0.008*"portchannel" + 0.008*"routemap" + 0.007*"bgp" + 0.006*"configure" + 0.006*"redistribute" + 0.006*"deadlock" + 0.006*"

In [38]:
X = re.sub(r'\w', ' ', X)

TypeError: expected string or bytes-like object